This notebook is not really cleaned up, but it was used to run a sensitivity analysis on the differing population methodologies for the Climate Cabinet IRA map.

In [3]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from statsmodels

In [1]:
ALBERS = 9822
NAD = 4269

DATA_FOLDER = '../data/'
GEOGRAPHIES_FOLDER = DATA_FOLDER + "clean/"

# Census data
BLOCK_GROUPS_CENTROID = DATA_FOLDER + 'CenPop2020_Mean_BG.txt'
BLOCK_GROUPS = DATA_FOLDER + 'census_block_group_pop.geoparquet'
TRACTS = DATA_FOLDER + 'census_tract_pop.geoparquet'
TRACTS_CENTROID = DATA_FOLDER + 'CenPop2020_Mean_TR.txt'

# Targets
RURAL_COOPS = DATA_FOLDER + 'rural_coops.geoparquet' # abnormal
MUNICIPAL = DATA_FOLDER + 'municipal_utils.geoparquet' # abnormal
COUNTIES = DATA_FOLDER + 'county_clean.geoparquet' # should follow tracts

# Bonus territories
DISTRESSED_COMMUNITIES = DATA_FOLDER + 'dci_clean.geoparquet' # abnormal
LOW_INCOME = DATA_FOLDER + 'low_income_tracts_filtered.geoparquet' # should follow tracts
J40 = DATA_FOLDER + 'justice40.geoparquet' # should follow tracts
FOSSIL = DATA_FOLDER + 'ffe.geoparquet' # could be a census tract or a metropolitan statistical area
COAL = DATA_FOLDER + 'coal_closure.geoparquet' # could be a census tract or a metropolitan statistical area


In [4]:
gdf_tracts = gpd.read_parquet(TRACTS).to_crs(ALBERS)

In [5]:
gdf_blocks = gpd.read_parquet(BLOCK_GROUPS).to_crs(ALBERS)

In [6]:
df_tracts_c = pd.read_csv(TRACTS_CENTROID)
df_tracts_c['geometry'] = df_tracts_c.apply(lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1)
gdf_tracts_c = gpd.GeoDataFrame(df_tracts_c)
gdf_tracts_c.crs = NAD
gdf_tracts_c = gdf_tracts_c.to_crs(ALBERS)

In [7]:
df_blocks_c = pd.read_csv(BLOCK_GROUPS_CENTROID)
df_blocks_c['geometry'] = df_blocks_c.apply(lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1)
gdf_blocks_c = gpd.GeoDataFrame(df_blocks_c)
gdf_blocks_c.crs = NAD
gdf_blocks_c = gdf_blocks_c.to_crs(ALBERS)

In [17]:
# Abnormal geographies: distressed communities, coops, municipal electric utilities
# What about energy communities?
# Do counties also as a sanity check
# Should follow tracts: J40, low income
gdf_dci = gpd.read_parquet(DISTRESSED_COMMUNITIES).to_crs(ALBERS)
gdf_li = gpd.read_parquet(LOW_INCOME).to_crs(ALBERS)
gdf_j40 = gpd.read_parquet(J40).to_crs(ALBERS)
gdf_ffe = gpd.read_parquet(FOSSIL).to_crs(ALBERS)
gdf_coal = gpd.read_parquet(COAL).to_crs(ALBERS)
gdf_mun = gpd.read_parquet(MUNICIPAL).to_crs(ALBERS)
gdf_county = gpd.read_parquet(COUNTIES).to_crs(ALBERS)
gdf_coops = gpd.read_parquet(RURAL_COOPS).to_crs(ALBERS)


In [18]:
len(gdf_dci), len(gdf_mun), len(gdf_coops)

(5212, 1938, 812)

In [9]:
gdf_dci.head(2)

,zip_code,metro_area,city,county,state,state_abbr,region,cty_type,total_pop,dist_score,quintile,geoid,area_land,area_water,lat,lon,geometry,simple_boundary
0,1009,"Springfield, MA","Palmer Town, MA","Hampden County, Massachusetts",Massachusetts,MA,Northeast,Small urban,649,82.911854,5,1009.0,1806483.0,0.0,+42.2113859,-072.3418086,"POLYGON ((-3749787.975 3788590.546, -3749625.5...","POLYGON ((-72.35284 42.20543, -72.34296 42.218..."
1,1040,"Springfield, MA","Holyoke, MA","Hampden County, Massachusetts",Massachusetts,MA,Northeast,Small urban,40161,91.868452,5,1040.0,54816657.0,4226508.0,+42.2116578,-072.6424476,"POLYGON ((-3766852.767 3810651.635, -3766851.1...","POLYGON ((-72.69046 42.21306, -72.65654 42.227..."


In [10]:
gdf_mun.head(2)

,ID,NAME,ADDRESS,CITY,ST_Code,ZIP,TELEPHONE,TYPE,COUNTRY,NAICS_CODE,...,TRANS_MWH,CUSTOMERS,YEAR,Shape__Are,Shape__Len,State,area,Type_1,geometry,simple_boundary
0,1000,CITY OF AUGUSTA - (AR),NOT AVAILABLE,NOT AVAILABLE,AR,NOT AVAILABLE,NOT AVAILABLE,MUNICIPAL,USA,2211,...,-999999,916,2020,0.152271,2.625553,Arkansas,2.308823e+09,Municipal/Public Utility,"POLYGON ((-5278797.041 4757879.683, -5278746.4...","POLYGON ((-91.34713 35.43899, -91.25296 35.445..."
1,1002,CITY OF AURELIA - (IA),NOT AVAILABLE,NOT AVAILABLE,IA,NOT AVAILABLE,NOT AVAILABLE,MUNICIPAL,USA,2211,...,-999999,518,2020,0.164284,1.639697,Iowa,2.771736e+09,Municipal/Public Utility,"POLYGON ((-4695466.747 5466293.520, -4695194.8...","POLYGON ((-95.85936 42.90908, -95.38814 42.909..."


In [11]:
gdf_county.head(2)

,STATEFP,COUNTYFP,County,ALAND,AWATER,INTPTLAT,INTPTLON,State,geometry,simple_boundary
0,31,039,Cuming County,1477645345,10690204,+41.9158651,-096.7885168,Nebraska,"POLYGON ((-4825667.406 5513810.893, -4825582.0...","POLYGON ((-97.01952 42.00410, -97.01936 42.090..."
1,53,069,Wahkiakum County,680976231,61568965,+46.2946377,-123.4244583,Washington,"POLYGON ((-4892810.695 7670833.790, -4891600.9...","POLYGON ((-123.43639 46.23820, -123.47964 46.2..."


In [12]:
gdf_blocks.head(2)

,STATEFP_x,COUNTYFP_x,TRACTCE_x,BLKGRPCE_x,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,STATEFP_y,COUNTYFP_y,TRACTCE_y,BLKGRPCE_y,POPULATION,LATITUDE,LONGITUDE
0,40,135,030201,1,401350302011,Block Group 1,G5030,S,101177851,696261,+35.5958901,-094.9482992,"POLYGON ((-5415010.265 5071763.046, -5414947.6...",40,135,30201,1,1395,35.602814,-94.976367
1,40,135,030101,1,401350301011,Block Group 1,G5030,S,45719893,485726,+35.4855356,-094.8413282,"POLYGON ((-5417259.558 5049917.863, -5417167.4...",40,135,30101,1,903,35.505339,-94.811951


### Test that Methodologies are the Same

In [37]:
effect_size = 0.1
alpha = 0.05
power = 0.9

# Calculate sample size for paired t-test
sample_size = smp.TTestPower().solve_power(effect_size=effect_size, 
                                           alpha=alpha, 
                                           power=power, 
                                           alternative='two-sided')

print(f"Required sample size: {sample_size:.2f}")

Required sample size: 1052.66


In [32]:
sample_size = 1100
area = []
centroid = []
for idx, row in gdf_dci.sample(n=sample_size).iterrows():
    # print(f"========Coal Tract ID: {row.TractID}========")
    # print(f"========Utility Name: {row.NAME}========")
    # print(f"========County: {row.County}========")
    # print(f"========DCI Name: {row.geoid}========")

    # Calculate using area allocation for tract
    # gdf_tracts['intersections'] = gpd.GeoSeries(gdf_tracts['geometry'].intersection(row.geometry))
    # result = gdf_tracts.loc[~gdf_tracts['intersections'].is_empty, ['total_pop', 'intersections', 'geometry']]
    # result['allocated_pop'] = result.total_pop * result.intersections.area/result.geometry.area
    # tract_area_pop = result.allocated_pop.sum()
    # area.append(tract_area_pop)
    # # print(f"Tract Area: {tract_area_pop}")

    # # Calculate using tract population weighted centroid
    # gdf_tracts_c['is_within'] = gdf_tracts_c['geometry'].apply(lambda geom: geom.within(row.geometry))
    # result = gdf_tracts_c[gdf_tracts_c['is_within']]
    # tract_centroid_pop = result.POPULATION.sum()
    # # print(f"Tract Centroid: {tract_centroid_pop}")

    # Calculate using block group area
    gdf_blocks['intersections'] = gpd.GeoSeries(gdf_blocks['geometry'].intersection(row.geometry))
    result = gdf_blocks.loc[~gdf_blocks['intersections'].is_empty, ['POPULATION', 'intersections', 'geometry']]
    result['allocated_pop'] = result.POPULATION * result.intersections.area/result.geometry.area
    block_area_pop = result.allocated_pop.sum()
    # print(f"Census Block Area: {tract_area_pop}")
    area.append(block_area_pop)

    # Calculate using block group weighted centroid
    gdf_blocks_c['is_within'] = gdf_blocks_c['geometry'].apply(lambda geom: geom.within(row.geometry))
    result = gdf_blocks_c[gdf_blocks_c.is_within]
    block_centroid_pop = result.POPULATION.sum()
    # print(f"Block Centroid: {block_centroid_pop}")
    centroid.append(block_centroid_pop)

In [33]:
centroid

[22827,
 3866,
 1377,
 8793,
 795,
 1472,
 3879,
 5451,
 636,
 13317,
 775,
 4890,
 5729,
 698,
 5024,
 1460,
 2296,
 19054,
 7209,
 2036,
 11783,
 1213,
 4157,
 0,
 3123,
 676,
 9237,
 16018,
 1034,
 4552,
 985,
 3149,
 2126,
 1518,
 978,
 42163,
 13757,
 2906,
 0,
 2964,
 1493,
 1444,
 24607,
 8851,
 21751,
 18278,
 2571,
 0,
 7600,
 3868,
 1391,
 6870,
 55419,
 15075,
 8586,
 28190,
 616,
 533,
 35164,
 14537,
 4757,
 41449,
 1498,
 11170,
 32763,
 29322,
 2374,
 6408,
 9305,
 20578,
 24464,
 5691,
 13588,
 5408,
 18609,
 3159,
 15496,
 11670,
 7837,
 0,
 8436,
 2822,
 14540,
 1546,
 0,
 1940,
 5072,
 5007,
 661,
 7684,
 5898,
 3665,
 25427,
 2183,
 2000,
 3076,
 2648,
 1610,
 573,
 24596,
 15616,
 22910,
 890,
 12655,
 10613,
 3622,
 31276,
 2782,
 10840,
 4116,
 9648,
 5416,
 2006,
 17262,
 1221,
 2589,
 2467,
 1619,
 0,
 2098,
 7119,
 20510,
 1175,
 11362,
 749,
 40529,
 1296,
 7894,
 50379,
 13270,
 2634,
 9333,
 5176,
 8926,
 3937,
 36319,
 1010,
 2698,
 3372,
 1127,
 2528,
 78

In [34]:
area

[20202.889488450288,
 6591.81626176692,
 940.0756500003961,
 8398.089975880615,
 1507.9395444371662,
 1671.4183964113165,
 4184.585450562567,
 5880.322469581605,
 1201.5869146034513,
 14176.793672491372,
 848.1355927804423,
 5154.32039612241,
 5682.271562527868,
 390.4963407207979,
 8065.833552112074,
 3035.1819614661326,
 1777.3632949020919,
 18231.144170312018,
 7210.419198152427,
 2070.7519296243136,
 11065.909370758196,
 1001.8111537192389,
 4211.4191069902245,
 656.8316547966778,
 2918.1788237679357,
 1484.7559590701235,
 9017.469920929905,
 14547.983944148997,
 823.9698517225816,
 4310.279896370388,
 662.9392248756787,
 2928.0508355994734,
 4628.841451504277,
 1130.1119739408714,
 1944.7455249176014,
 42207.70267200461,
 14649.091749427549,
 2361.168722986308,
 375.36374840246697,
 2456.7019742372804,
 1870.2119844904012,
 909.7071778524745,
 24746.69484315642,
 12880.464095927633,
 20082.604980934888,
 19337.5238777826,
 3447.794002089347,
 833.7188617715317,
 7567.694741636627,

In [35]:
import scipy.stats as stats

t_statistic, p_value = stats.ttest_rel(area, centroid)

print(f"T-statistic: {t_statistic}, P-value: {p_value}")

T-statistic: -0.3577212379467644, P-value: 0.7206206258238588


In [49]:
gdf_blocks.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE,geometry
0,1,1,20100,1,575,32.464466,-86.486302,POINT (-5355062.078 4200460.527)
1,1,1,20100,2,1200,32.482744,-86.486741,POINT (-5353306.657 4201527.272)
2,1,1,20200,1,974,32.478370,-86.474486,POINT (-5353145.986 4200271.135)
3,1,1,20200,2,1081,32.466065,-86.470884,POINT (-5354169.566 4199280.412)
4,1,1,20300,1,2377,32.477855,-86.459855,POINT (-5352496.506 4199036.946)


### Old Code Below here

In [11]:
dci_comm = gdf_dci.iloc[4].geometry

In [31]:
print(result.to_crs(9822).crs.axis_info[0].unit_name)

metre


In [28]:
result.to_crs(9822).geometry.area

36701    1.212694e+06
36702    4.534280e+05
36703    8.698111e+05
36704    1.657655e+06
36716    1.332822e+06
36717    8.357953e+05
36718    8.629863e+05
36719    1.128168e+06
36720    5.058099e+06
36721    5.955474e+05
36759    5.333531e+06
36780    8.157156e+06
dtype: float64

In [38]:
# Compute the intersections with every geometry in gdf2
gdf_tracts['intersections'] = gpd.GeoSeries(gdf_tracts['geometry'].intersection(dci_comm)).to_crs(9822)
result = gdf_tracts.loc[~gdf_tracts['intersection'].is_empty, ['total_pop', 'intersections', 'geometry']]
result = result.to_crs(9822) # Alber Equal Area
result['allocated_pop'] = result.total_pop * result.intersections.area/result.geometry.area

# Filter out the empty geometries
# non_empty_intersections = intersections[~intersections.is_empty]


In [39]:
result.allocated_pop

36701    3.263590e-10
36702    7.920601e+02
36703    1.884389e+03
36704    2.705071e+01
36716    1.415373e+02
36717    2.620588e+03
36718    3.541617e+03
36719    2.574435e+03
36720    1.455906e-10
36721    0.000000e+00
36759    8.149060e-11
36780    1.871436e-10
Name: allocated_pop, dtype: float64

In [59]:
result.allocated_pop.sum()

11581.677226702635

Now need to do this with the population weighted centroid

In [46]:
from shapely.geometry import Point

In [44]:
df_tracts = pd.read_csv(TRACTS_CENTROID)

In [45]:
df_tracts.head()

,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE
0,1,1,20100,1775,32.476823,-86.486599
1,1,1,20200,2055,32.471897,-86.472591
2,1,1,20300,3216,32.474583,-86.459254
3,1,1,20400,4246,32.468656,-86.442995
4,1,1,20501,4322,32.451532,-86.424422


In [47]:
df_tracts['geometry'] = df_tracts.apply(lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1)

In [54]:
gdf_tracts_centroid = gpd.GeoDataFrame(df_tracts)

In [56]:
gdf_tracts_centroid.crs = 9822

In [57]:
gdf_tracts_centroid.head()

,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE,geometry
0,1,1,20100,1775,32.476823,-86.486599,POINT (-86.487 32.477)
1,1,1,20200,2055,32.471897,-86.472591,POINT (-86.473 32.472)
2,1,1,20300,3216,32.474583,-86.459254,POINT (-86.459 32.475)
3,1,1,20400,4246,32.468656,-86.442995,POINT (-86.443 32.469)
4,1,1,20501,4322,32.451532,-86.424422,POINT (-86.424 32.452)


In [60]:
gdf_tracts_centroid['is_within'] = gdf_tracts_centroid['geometry'].apply(lambda geom: geom.within(dci_comm))

In [61]:
gdf_tracts_centroid.head()

,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE,geometry,is_within
0,1,1,20100,1775,32.476823,-86.486599,POINT (-86.487 32.477),False
1,1,1,20200,2055,32.471897,-86.472591,POINT (-86.473 32.472),False
2,1,1,20300,3216,32.474583,-86.459254,POINT (-86.459 32.475),False
3,1,1,20400,4246,32.468656,-86.442995,POINT (-86.443 32.469),False
4,1,1,20501,4322,32.451532,-86.424422,POINT (-86.424 32.452),False


In [62]:
results_centroid = gdf_tracts_centroid[gdf_tracts_centroid['is_within']]

In [65]:
results_centroid.head()

,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE,geometry,is_within
36702,25,13,801102,1155,42.099237,-72.583094,POINT (-72.583 42.099),True
36703,25,13,801200,2726,42.109322,-72.583787,POINT (-72.584 42.109),True
36717,25,13,801901,3954,42.094700,-72.571346,POINT (-72.571 42.095),True
36718,25,13,801902,3556,42.102610,-72.577042,POINT (-72.577 42.103),True
36719,25,13,802000,2736,42.095008,-72.579091,POINT (-72.579 42.095),True


In [66]:
results_centroid.POPULATION.sum()

14127

In [67]:
# Do this for some counties just to make sure you're not dumb